In [1]:
import random
import pandas as pd
from nltk.stem.snowball import RussianStemmer

In [2]:
df = pd.read_csv('dataset/dataset_0.csv', index_col=False)
df.head()

,WORD,GROUND_TRUTH
0,отделение_NOUN,1.0
1,секция_NOUN,1.0
2,палата_NOUN,1.0
3,богадельня_NOUN,1.0
4,особа_NOUN,1.0


In [3]:
df.tail()

,WORD,GROUND_TRUTH
66,ссора_NOUN,0.0
67,графиня_NOUN,0.0
68,археолог_NOUN,0.0
69,похищение_NOUN,0.0
70,влажный_ADJ,0.0


In [40]:
len(df)

71

In [27]:
words = df['WORD'].tolist()
labels = df['GROUND_TRUTH'].tolist()
pairs = list(zip(words,labels))
print(len(pairs))
# for word in used:
#     pairs.remove(word)
# print(len(pairs))
sample_words = random.sample(pairs, 5)
#sample_words = [word.split('_')[0] for word in sample_words]
for pair in sample_words:
    print(pair)

71
('переулок_NOUN', 0.0)
('машина_NOUN', 1.0)
('особа_NOUN', 1.0)
('богадельня_NOUN', 1.0)
('молодец_NOUN', 1.0)


In [5]:
corpora1 = pd.read_csv('./macro/corpora/pre-soviet.csv', index_col='ID')
corpora1.head()

/home/julia/.local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,LEMMAS,RAW
ID,,
0,А Корнилович .,А.О. Корнилович.
1,"письмо ваш превосходительство , милостивый гос...","Письмо Ваше Превосходительство, Милостивый Гос..."
2,пользоваться благосклонный позволение обращать...,Пользуясь благосклонным позволением обращаться...
3,"в ожидание книга , какой угодно быть ваш прево...","В ожидании книг, какие угодно было Вашему Прев..."
4,"они состоять из стихотворение шекспир , Шиллер...","Они состоят из стихотворений Шекспира, Шиллера..."


In [6]:
corpora2 = pd.read_csv('./macro/corpora/soviet.csv', index_col='ID')
corpora2.head()

,LEMMAS,RAW
ID,,
0,Морозов а. .,Морозов А..
1,"долой "" вершининщина "" !","Долой ""вершининщину""!"
2,"уже весь кооператор губерния , вероятно , знат...","Уже все кооператоры губернии, вероятно, знают ..."
3,начало расследование дело быть класть заметка ...,Начало расследованию дела было положено заметк...
4,"киселев , к который приходить какой-то девушка...","Киселева, к которому пришла какая-то девушка и..."


In [7]:
# len(corpora2)

8068459

In [8]:
# test = corpora2.drop_duplicates()
# len(test)

7046212

In [7]:
stemmer = RussianStemmer()

In [8]:
def get_contexts(word, corpora):
    samples = list()
    stem = stemmer.stem(word)
    previous = list()
    nextt = list()
    
    for idx, lemmas, raw in corpora[['LEMMAS', 'RAW']].itertuples():
        if word in lemmas.lower().split() and raw not in samples:
            samples.append(raw)
            previous.append(corpora.loc[idx-1, 'RAW'])
            nextt.append(corpora.loc[idx+1, 'RAW'])
      
    for i, sentence in enumerate(samples):
            tokens = sentence.split()
            for k, token in enumerate(tokens):
                if token.lower().startswith(stem):
                    tokens[k] = '<b><i>'+token+'</i></b>'
            samples[i] = ' '.join(tokens)
            
    return list(zip(samples, previous, nextt))

In [41]:
results = pd.DataFrame(columns=['INPUT:word', 'INPUT:type', 'INPUT:old', 'INPUT:old_prev', 'INPUT:old_next', 
                                'INPUT:new', 'INPUT:new_prev', 'INPUT:new_next', 'GOLDEN:result', 'HINT:text'])

for word in sample_words:
    old_samples = get_contexts(word[0].split('_')[0], corpora1)
    new_samples = get_contexts(word[0].split('_')[0], corpora2)
    
    divisible=60
    if len(old_samples) > 60 and len(new_samples) > 60:
        old_samples_fin = random.sample(old_samples, 60)
        new_samples_fin = random.sample(new_samples, 60)
    else:
        num = min(len(old_samples), len(new_samples))
        divisible = num - (num % 3)
        old_samples_fin = random.sample(old_samples, divisible)
        new_samples_fin = random.sample(new_samples, divisible)

    old_samples_div = [old_samples_fin[i:i + int(divisible/3)] for i in range(0, len(old_samples_fin), int(divisible/3))]
    new_samples_div = [new_samples_fin[i:i + int(divisible/3)] for i in range(0, len(new_samples_fin), int(divisible/3))]
    
    for pair in zip(old_samples_div[0], old_samples_div[1]):
        row = [word[0].split('_')[0], 'EARLIER', pair[0][0], pair[0][1], pair[0][2], 
               pair[1][0], pair[1][1], pair[1][2], None, None]
        results.loc[len(results)] = row
        
    for pair in zip(new_samples_div[0], new_samples_div[1]):
        row = [word[0].split('_')[0], 'LATER', pair[0][0], pair[0][1], pair[0][2], 
               pair[1][0], pair[1][1], pair[1][2], None, None]
        results.loc[len(results)] = row
    
    for pair in zip(old_samples_div[2], new_samples_div[2]):
        row = [word[0].split('_')[0], 'COMPARE', pair[0][0], pair[0][1], pair[0][2], 
               pair[1][0], pair[1][1], pair[1][2], None, None]
        results.loc[len(results)] = row

In [42]:
results = results.sample(frac=1).reset_index(drop=True)
results.head(30)

,INPUT:word,INPUT:type,INPUT:old,INPUT:old_prev,INPUT:old_next,INPUT:new,INPUT:new_prev,INPUT:new_next,GOLDEN:result,HINT:text
0,молодец,COMPARE,"-- Ай да Рыжик, <b><i>молодец!</i></b> -- прив...","Сам он, довольный и счастливый, был похож на м...","-- Ну, садись, земляк, обедать, а после доскаж...","Старый есаул Витошнов, человек со скуластым ли...","Как завладеем денежками, да довольствием, да з...",..,None,None
1,переулок,COMPARE,5. [Б. Троицкий пер.,"Александрович] Grenoble, Isre, 7 rue de la Lib...","(Новопроектированный, ехать по Троицкой ул.)] ...","Вдруг, с ребяческим наслаждением, я заметил, ч...","Помню, какое восхищение он вызывал во мне зимн...",Менее интересным был другой перерыв в одной из...,None,None
2,машина,COMPARE,"Я уверен, что всякая заминка в деле флота буде...","Не сетуйте, господа, если и правительство выск...","Господа, в деле воссоздания нашего морского мо...","Поэтому все говорит за постройку простейших ""б...","Постройка ""багги"" (как, впрочем, и любого друг...",А именно такие соревнования подготовят молодых...,None,None
3,богадельня,EARLIER,"Противно сему, когда и хозяин стояльца изобиди...",8.,9.,После того как Курбатов был переведен вице-губ...,"Мы видели, что первым делом Петра было уничтож...","В это время страшного труда для тех, которые о...",None,None
4,особа,EARLIER,В моем уме стремление к теоретическим исследов...,"Даровав мне жизнь, ты в то же время позаботилс...","По-видимому, именно в предвидении этого случая...",-- Кто эта <b><i>особа</i></b> -- спрашивали в...,"Обе вполне прекрасны, неподдельно свежи, непод...",-- Как! вы не знаете.,None,None
5,богадельня,EARLIER,Иван Алексеевич нашелся и тут: -- Не думай люб...,"Александр отказался, в свою очередь.",Александр был в хороших отношениях с Дмитрием ...,Каторжная Дербинская <b><i>богадельня</i></b> ...,Словно куча навоза догнивает на солнце.,А.П. Чехов.,None,None
6,машина,EARLIER,Просим товарищей обратить серьезное внимание н...,П.П. Сойкина (забастовка началась 9 апреля) тв...,"Они верно забыли, что вышли из среды тех же ра...",10-го числа в первой рас пущена <b><i>машина</...,Май.,С 1-го и до 11-го чисел сего месеца погода сто...,None,None
7,переулок,LATER,Ляпис обитал в доме 9 по Казарменному переулку...,Через пять минут раскачивающийся трамвай уноси...,Ляпис носил капитанскую фуражку с гербом Непту...,Она жила в Газетном переулке.,"Когда Никритина уезжала в Киев, из какой-то те...",Путь к Александровскому шел по Никитской мимо ...,None,None
8,особа,LATER,Она росла премиленькой девочкой-резвушкой и ша...,Сестра Катя Вторая моя сестра -- Екатерина по ...,Детей вообще привлекает изящное или яркое проя...,Между тем все обошлось: раненым нацепили крест...,"Она не запомнила даже лица царского, -- все гл...",-- Никогда не сомневался в моих славных кубанц...,None,None
9,молодец,LATER,"-- <b><i>Молодец,</i></b> что догадался.","-- Правильно, -- обрадовалась я.","Действительно, никому ты такой не нужен.","В столовую, на паркет, молодцы-плотники, в род...","Погода разгулялась, большое солнце.",Ляжешь на тюк -- и дышишь: яблочными садами па...,None,None


In [43]:
len(results)

300

In [44]:
results.to_csv('test_sample_0.tsv', sep='\t', index=False)